In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# Load the ratings file
ratings = pd.read_csv("ratings_small.csv")

# Load the movie metadata
metadata = pd.read_csv("movies_metadata.csv", sep=',', quotechar='"', engine='python', parse_dates=['release_date'], date_format='%Y-%m-%d', on_bad_lines='skip')

# Filter movies with numeric IDs only
metadata = metadata[metadata['id'].apply(lambda x: x.isnumeric())]
metadata['id'] = metadata['id'].astype(int)

# Merge ratings with metadata
merged = pd.merge(ratings, metadata, left_on='movieId', right_on='id')

In [20]:
# Create a pivot table: rows = users, columns = movie titles, values = ratings
user_movie_matrix = merged.pivot_table(index='userId', columns='title', values='rating')

# Fill NaN with 0 (assume unrated movies as 0)
user_movie_matrix.fillna(0, inplace=True)

In [21]:
# Compute cosine similarity between movies
item_similarity = cosine_similarity(user_movie_matrix.T)

# Create a DataFrame to store similarity scores
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

In [22]:
def recommend_movies(movie_title, sim_df, top_n=5):
    if movie_title not in sim_df.columns:
        return ["Movie not found in dataset."]

    # Get similar movies, sort and return top_n
    similar_movies = sim_df[movie_title].sort_values(ascending=False)[1:top_n+1]
    return list(similar_movies.index)

In [26]:
print(list(user_movie_matrix.columns))

['!Women Art Revolution', "'Gator Bait", "'Twas the Night Before Christmas", '...And God Created Woman', '00 Schneider - Jagd auf Nihil Baxter', '10 Items or Less', '10 Things I Hate About You', '10,000 BC', "11'09''01 - September 11", '12 Angry Men', '13 Tzameti', '1408', '15 Minutes', '16 Blocks', '1900', '1942: A Love Story', '1984', '2 Days in Paris', '20,000 Leagues Under the Sea', '2001: A Space Odyssey', '2010', '2046', '2061 - Un anno eccezionale', '21 Grams', '24 Hour Party People', '25th Hour', '28 Days Later', '28 Weeks Later', '29th Street', '2:37', '3 Ninjas: High Noon at Mega Mountain', '3-Iron', '300', '33 Scenes from Life', '4 Months, 3 Weeks and 2 Days', '40 Days and 40 Nights', '42nd Street', '48 Hrs.', '5 Card Stud', '5 Days of War', '50 First Dates', '54', '7 Virgins', '8 Mile', '8 Women', '88 Minutes', '8:46', '8½', '9 Songs', '95 Miles to Go', '964 Pinocchio', '99 francs', 'A Beautiful Mind', 'A Bridge Too Far', 'A Brief History of Time', 'A Chef in Love', 'A Chor

In [36]:
recommended = recommend_movies("'Gator Bait", item_sim_df)

print("Recommended Movies:")
for movie in recommended:
    print("🎬", movie)

Recommended Movies:
🎬 Murder by Death
🎬 In the Darkness of the Night
🎬 Exit
🎬 Blood Ties
🎬 'Gator Bait
